## Simple RAG에서 Chunk size 선택하기

RAG 파이프라인에서 **적절한 Chunk 크기를 설정하는 것**은 검색 정확도를 높이는 데 매우 중요합니다. 핵심은 **검색 성능과 응답 품질 사이의 균형**을 맞추는 것입니다.

#### 이 노트북의 주요 단계는 다음과 같습니다:
1. PDF에서 텍스트를 추출합니다.
2. 텍스트를 여러 크기의 Chunk로 나눕니다.
3. 각 Chunk에 대해 Embedding을 생성합니다.
4. 쿼리에 대해 관련 있는 Chunk들을 검색합니다.
5. 검색된 Chunk를 바탕으로 응답을 생성합니다.
6. 생성된 응답의 **사실성(faithfulness)**과 **관련성(relevancy)**을 평가합니다.
7. Chunk size 별 평가를 비교합니다.

## 환경 설정하기
필요한 라이브러리를 가져옵니다.

In [ ]:
import os
import numpy as np
from dotenv import load_dotenv
from tqdm import tqdm


# .env 파일 로드
load_dotenv()

API_KEY = os.environ.get('api_key')

## OpenAI API 클라이언트 설정하기

In [ ]:
from openai import OpenAI

client_openai = OpenAI(api_key = API_KEY)

## PDF 파일에서 텍스트 추출하기
RAG를 구현하려면 먼저 텍스트 데이터 소스가 필요합니다. 저는 gemini를 이용해 pdf에서 텍스트를 추출하는 방식을 사용합니다.  
만약 txt 형태로 파일이 존재한다면 `load_text_file` 함수를 사용하면됩니다.

In [ ]:
import google.generativeai as genai

def extract_text_from_pdf(pdf_path):
    # API 키 설정
    genai.configure(api_key=gemini_API_KEY)
    client = genai.GenerativeModel('gemini-2.0-flash-lite')

    # PDF 파일 업로드
    with open(pdf_path, "rb") as file:
        file_data = file.read()


    prompt = "Extract all text from the provided PDF file."
    response = client.generate_content([
        {"mime_type": "application/pdf", "data": file_data},
        prompt
    ],generation_config={
            "max_output_tokens": 8192  # 최대 출력 토큰 수 설정 (예: 8192 토큰, 약 24,000~32,000자)
    })
    return response.text

In [ ]:
# 이미 text 파일로 저장되어 있다면 load_text_file 함수를 사용하면 됩니다.
def load_text_file(pdf_path):

    # text 파일 로드
    with open(pdf_path, "r", encoding="utf-8") as txt_file:
        text = txt_file.read()

    return text

txt_path = "./data_creation/pdf_data/(1) 2024 달라지는 세금제도.txt"

extracted_text = load_text_file(txt_path)
print(extracted_text[:500])

## 추출된 텍스트 청크 분할
텍스트를 추출한 뒤에는 검색 정확도를 높이기 위해 조금씩 겹치도록 나눠서 작은 단위로 분할(chunk)합니다.  
이번 노트북에서는 여러 사이즈로 chunk를 분할합니다.

In [ ]:
def chunk_text(text, n, overlap):
    """
    주어진 텍스트를 겹치는 n개의 문자 세그먼트로 청크합니다.

    Args:
    text (str): 청크할 텍스트
    n (int): 각 청크의 문자 길이
    overlap (int): 청크 간 겹치는 문자 길이

    Returns:
    List[str]: 청크된 텍스트 리스트
    """
    chunks = []  ## 청크된 텍스트를 저장할 리스트
    
    # overlap만큼 겹치도록 text를 n의 길이로 chunking
    for i in range(0, len(text), n - overlap):
        chunks.append(text[i:i + n])

    return chunks  

# 다양한 크기의 청크 사이즈를 정의합니다.
chunk_sizes = [256, 512, 1024]

# 각 청크 크기에 대한 텍스트 청크를 저장할 딕셔너리 생성
text_chunks_dict = {size: chunk_text(extracted_text, size, size // 5) for size in chunk_sizes}

# 각 청크 크기에 대한 청크 개수 출력
for size, chunks in text_chunks_dict.items():
    print(f"Chunk Size: {size}, Number of Chunks: {len(chunks)}")

## 분할된 청크의 임베딩 생성
임베딩은 텍스트를 숫자 벡터로 변환하여 효율적인 유사도 검색을 가능하게 합니다.

In [ ]:
import torch
from sentence_transformers import SentenceTransformer
def create_embeddings(embedding_model, texts, device='cuda', batch_size=16):
    """
    SentenceTransformer 모델을 사용하여 텍스트의 임베딩 생성

    Args:
        embedding_model: 임베딩을 생성할 SentenceTransformer 모델입니다.
        texts (list): 임베딩을 생성할 입력 텍스트 리스트입니다.
        device (str): 모델을 실행할 장치 ('cuda' for GPU, 'cpu' for CPU).
        batch_size (int): 한번에 처리할 텍스트의 개수

    Returns:
        np.ndarray: 임베딩
    """
    # 모델이 지정된 장치에 있는지 확인합니다.
    embedding_model = embedding_model.to(device)
    
    # 지정된 배치 크기로 임베딩을 생성합니다.
    embeddings = embedding_model.encode(
        texts,
        device=device,
        batch_size=batch_size,  # 메모리 사용량을 줄이기 위해 더 작은 배치 크기를 사용합니다.
        show_progress_bar=True  # 인코딩 진행 상태를 모니터링하기 위해 진행 표시줄 출력
    )
    
    return embeddings

# GPU 사용 가능 여부를 확인합니다.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# 모델을 로드합니다.
model = "BAAI/bge-m3"
embedding_model = SentenceTransformer(model)

# 각 청크 크기에 대한 임베딩을 생성합니다.
chunk_embeddings_dict = {size: create_embeddings(embedding_model, chunks) for size, chunks in tqdm(text_chunks_dict.items(), desc="Generating Embeddings")}

## Semantic Search
코사인 유사도를 구현하여 사용자 쿼리에 가장 관련성이 높은 텍스트 청크를 찾습니다.

In [ ]:
def cosine_similarity(vec1, vec2):
    """
    두 벡터 간의 코사인 유사도를 계산합니다.

    Args:
    vec1 (np.ndarray): 첫 번째 벡터입니다.
    vec2 (np.ndarray): 두 번째 벡터입니다.

    Returns:
    float: 두 벡터 간의 코사인 유사도입니다.
    """
    # 두 벡터의 내적을 계산하고 두 벡터의 크기의 곱으로 나눕니다.
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [ ]:
def retrieve_relevant_chunks(query, text_chunks, chunk_embeddings, k=5):
    """
    tok-k의 가장 유사한 청크를 검색
    
    Args:
    query (str): 사용자 쿼리
    text_chunks (List[str]): 텍스트 청크 리스트
    chunk_embeddings (List[np.ndarray]): 텍스트 청크 임베딩 리스트
    k (int): 상위 k개의 관련 텍스트 청크 반환
    
    Returns:
    List[str]: 가장 유사한 상위 k개의 텍스트 청크 리스트
    """
    # 쿼리에 대한 임베딩 생성
    query_embedding = create_embeddings(embedding_model, [query])[0]
    
    # 쿼리 임베딩과 각 청크 임베딩 간의 코사인 유사도 계산
    similarities = [cosine_similarity(query_embedding, emb) for emb in chunk_embeddings]
    
    # 가장 유사한 상위 k개의 청크 인덱스 추출
    top_indices = np.argsort(similarities)[-k:][::-1]
    
    # 가장 유사한 상위 k개의 관련 텍스트 청크 반환
    return [text_chunks[i] for i in top_indices]

## 테스트

In [ ]:
import pandas as pd

# 평가 데이터 로드
df = pd.read_csv('./data_creation/rag_val_new_post.csv')
df.head()

In [ ]:
# 평가 데이터에서 첫 번째 쿼리 추출
query = df['query'][0]

# 각 chunk size 별 semantic search 수행
retrieved_chunks_dict = {size: retrieve_relevant_chunks(query, text_chunks_dict[size], chunk_embeddings_dict[size]) for size in chunk_sizes}

# chunk size = 256에 대한 검색된 청크 출력
print(retrieved_chunks_dict[256])

## 검색된 청크를 기반으로 response 생성하기

In [ ]:
def generate_response(system_prompt, query, retrieved_chunks, model_name='gpt-4.1-nona'):
    # Combine retrieved chunks into a single context string
    context = "\n".join([f"Context {i+1}:\n{chunk}" for i, chunk in enumerate(retrieved_chunks)])
    
    # Create the user prompt by combining the context and the query
    user_prompt = f"{context}\n\nQuestion: {query}"

    response = client_openai.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": system_prompt},# Define the system prompt for the AI assistant
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.1,
        top_p=0.9,
        max_tokens=1024,
    )
    # print(response.choices[0].message.content)

    return response

# 시스템 프롬프트
system_prompt = "당신은 제공된 Context에 기반하여 답변하는 AI 어시스턴트입니다. 답변이 컨텍스트에서 직접 도출될 수 없는 경우, 다음 문장을 사용하세요: '해당 질문에 답변할 충분한 정보가 없습니다.'"

# chunk size 별 응답 생성
ai_responses_dict = {size: generate_response(system_prompt, query,  retrieved_chunks_dict[size],'gpt-4.1-nano-2025-04-14') for size in chunk_sizes}

In [ ]:
# chunk size = 256에 대한 응답 출력
print(ai_responses_dict[256].choices[0].message.content)

# chunk size = 512에 대한 응답 출력
print(ai_responses_dict[512].choices[0].message.content)

## 생성 응답 평가하기
생성된 답변과 예상 답변을 비교하여 평가합니다. 답변 평가시에는 LLM을 사용합니다.

In [ ]:

# 평가 점수 시스템 상수 정의
SCORE_FULL = 1.0     # 완전히 일치하거나 만족스럽다
SCORE_PARTIAL = 0.5  # 부분적으로 일치하거나 만족스럽다
SCORE_NONE = 0.0     # 일치하지 않거나 만족스럽지 않다

In [ ]:
# 엄격한 평가 프롬프트 템플릿 정의
FAITHFULNESS_PROMPT_TEMPLATE = """
Evaluate the faithfulness of the AI response compared to the true answer.
User Query: {question}
AI Response: {response}
True Answer: {true_answer}

Faithfulness measures how well the AI response aligns with facts in the true answer, without hallucinations.

INSTRUCTIONS:
- Score STRICTLY using only these values:
    * {full} = Completely faithful, no contradictions with true answer
    * {partial} = Partially faithful, minor contradictions
    * {none} = Not faithful, major contradictions or hallucinations
- Return ONLY the numerical score ({full}, {partial}, or {none}) with no explanation or additional text.
"""

In [ ]:
RELEVANCY_PROMPT_TEMPLATE = """
Evaluate the relevancy of the AI response to the user query.
User Query: {question}
AI Response: {response}

Relevancy measures how well the response addresses the user's question.

INSTRUCTIONS:
- Score STRICTLY using only these values:
    * {full} = Completely relevant, directly addresses the query
    * {partial} = Partially relevant, addresses some aspects
    * {none} = Not relevant, fails to address the query
- Return ONLY the numerical score ({full}, {partial}, or {none}) with no explanation or additional text.
"""

In [ ]:
def evaluate_response(question, response, true_answer, model='gpt-4.1-mini'):
        """
        faithfulness와 relevancy을 기준으로 AI가 생성한 응답의 품질 평가

        Args:
        question (str): 사용자의 원래 질문
        response (str): 평가될 AI 생성 응답
        true_answer (str): 정답으로 사용된 실제 답변

        Returns:
        Tuple[float, float]: (faithfulness_score, relevancy_score)
        """
        # 평가 프롬프트 포맷팅
        faithfulness_prompt = FAITHFULNESS_PROMPT_TEMPLATE.format(
                question=question, 
                response=response, 
                true_answer=true_answer,
                full=SCORE_FULL,
                partial=SCORE_PARTIAL,
                none=SCORE_NONE
        )
        
        relevancy_prompt = RELEVANCY_PROMPT_TEMPLATE.format(
                question=question, 
                response=response,
                full=SCORE_FULL,
                partial=SCORE_PARTIAL,
                none=SCORE_NONE
        )

        faithfulness_response = client_openai.chat.completions.create(
        model=model,
        messages=[
                {"role": "system", "content": "You are an objective evaluator. Return ONLY the numerical score."},
                {"role": "user", "content": faithfulness_prompt}
                ],
        temperature=0.1,
        top_p=0.9,
        max_tokens=1024)

        relevancy_response = client_openai.chat.completions.create(
        model=model,
        messages=[
                {"role": "system", "content": "You are an objective evaluator. Return ONLY the numerical score."},
                {"role": "user", "content": relevancy_prompt}
                ],
        temperature=0.1,
        top_p=0.9,
        max_tokens=1024,
        )

        # 평가 점수 추출
        # 점수를 추출하고 파싱 오류 처리
        try:
                faithfulness_score = float(faithfulness_response.choices[0].message.content.strip())
        except ValueError:
                print("Warning: Could not parse faithfulness score, defaulting to 0")
                faithfulness_score = 0.0
                
        try:
                relevancy_score = float(relevancy_response.choices[0].message.content.strip())
        except ValueError:
                print("Warning: Could not parse relevancy score, defaulting to 0")
                relevancy_score = 0.0

        return faithfulness_score, relevancy_score

# 첫 번째 평가 데이터의 참고(정답) 답변
true_answer = df['generation_gt'][0]

# 청크 크기 256과 512에 대한 응답 평가
faithfulness, relevancy = evaluate_response(query, ai_responses_dict[256], true_answer, 'gpt-4.1-mini')
faithfulness2, relevancy2 = evaluate_response(query, ai_responses_dict[512], true_answer, 'gpt-4.1-mini')

# 평가 점수 출력
print(f"Faithfulness Score (Chunk Size 256): {faithfulness}")
print(f"Relevancy Score (Chunk Size 256): {relevancy}")

print(f"\n")

print(f"Faithfulness Score (Chunk Size 512): {faithfulness2}")
print(f"Relevancy Score (Chunk Size 512): {relevancy2}")